# TODO : create a parcellation from the average of the similatriy matrix,

In [1]:
from pathlib import Path
import numpy as np
import nibabel as nib 
import networkx as nx

from smoothing import smooth_surface_graph
from gradient import compute_gradients, compute_gradient_average, build_mesh_graph, \
                     save_gradient_mgh, load_gradient_mgh
from watershed import watershed_by_flooding, save_labels_mgh \
                    , load_labels_mgh
from visualization import visualize_brain_surface

In [2]:
# Path Run all subject lh on run 1

config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N20")
}

hemisphere = 'lh'
run = 2

surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
# Extract the Surface Mesh
coords, faces = nib.freesurfer.read_geometry(str(surface_path))
coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
graph = build_mesh_graph(faces)

# Output paths
output_gradient_dir = config["subjects_dir"] / "gradient"
(output_gradient_dir).mkdir(exist_ok=True, parents=True)
output_labels_dir = config["subjects_dir"] / "labels"
(output_labels_dir).mkdir(exist_ok=True, parents=True)



# Variables path
subject = f"{1:02d}"
subj_dir = config["subjects_dir"] / f"sub-{subject}"

# Define paths using pathlib
surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"


# If needed, the grey matter mask
# gm_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / f"{hemisphere}.ribbon.mgz"


print('all paths defined')

all paths defined


In [3]:
from nilearn.image import resample_img
def load_data_normalized(surf_fmri_path,
                         vol_fmri_path, 
                         brain_mask_path):

    # Load the surface data
    surf_fmri = nib.load(str(surf_fmri_path)).get_fdata().squeeze()
    surf_fmri_n = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)
    
    # Load the images using nibabel
    vol_fmri = nib.load(str(vol_fmri_path))
    mask_img = nib.load(brain_mask_path)

    # resample the mask to the right one
    mask_img = resample_img(
        mask_img,
        target_affine=vol_fmri.affine,
        target_shape=vol_fmri.get_fdata().shape[:-1],
        interpolation='nearest',
        force_resample=True
    )

    fmri_data = vol_fmri.get_fdata()
    mask_data = mask_img.get_fdata().astype(bool)  # Convert mask to boolean
    # Normalize the data
    vol_fmri = fmri_data[mask_data] 
    vol_fmri_n = (vol_fmri - np.mean(vol_fmri, axis=1, keepdims=True)) / np.std(vol_fmri, axis=1, keepdims=True)
    
    # Remove nans from the data
    surf_fmri_n = np.nan_to_num(surf_fmri_n).astype(np.float32)
    vol_fmri_n = np.nan_to_num(vol_fmri_n).astype(np.float32)
    return surf_fmri_n, vol_fmri_n

In [4]:
from sklearn.decomposition import PCA
def compute_similarty_matrix_PCA(vol_fmri_n,
                                 surf_fmri_n,
                                 n_components=17):
    """Compute the similarity matrix with a PCA dim reduction
    Args:
        vol_fmri_n (_type_): _description_
        surf_fmri_n (_type_): _description_
        n_components (int, optional): _description_. Defaults to 17.
    """
    # 4. Run PCA on the time series data
    n_components = 17  # Number of PCA components you want to extract
    pca = PCA(n_components=n_components)
    temporal_modes = pca.fit_transform(vol_fmri_n.T).astype(np.float32) # shape = (n_timepoints, n_components)
    
    # Correlation formula for normalized data
    corr_matrix = (surf_fmri_n @ temporal_modes)  / (surf_fmri_n.shape[1] - 1) # shape = (n_vertices, n_components)
    # Similarty matrix
    sim_matrix = np.corrcoef(corr_matrix, dtype=np.float32) # shape = (n_vertices, n_vertices)
    sim_matrix = np.nan_to_num(sim_matrix) # remove nans if any
    return sim_matrix

In [ ]:
# Full pipeline my way

# 1 : Compute the average similartiy matrix across each subjects
sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
print('sim_matrix_sum shape', sim_matrix_sum.shape)
for i in range(1,21):
    if i == 5:
        continue # Subject 5 is missing
    # Variables path
    subject = f"{i:02d}"
    subj_dir = config["subjects_dir"] / f"sub-{subject}"

    # Define paths using pathlib
    surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
    vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
    brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"

    # Load the data
    surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                    vol_fmri_path, 
                                                    brain_mask_path)
    
    sim_matrix_sum += compute_similarty_matrix_PCA(vol_fmri_n,
                                                   surf_fmri_n)

# Save the similarty matrix
sim_matrix_path = config["subjects_dir"] / f"similarity_matrix_allsub_run{run}_{hemisphere}.npy"
np.save(sim_matrix_path, sim_matrix_sum)
    
    
# 2 : Smooth the similatry matrix
sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

# 3 : Compute the gradient of the similarty matrix
gradients = compute_gradients(graph, sim_matrix_smoothed)
gradients_sum = gradients.sum(axis=1).astype(np.float32)

# 3.1 : Smooth the gradient
gradient_smoothed = smooth_surface_graph(graph, gradients_sum, iterations=10)
gradients_path = config["subjects_dir"] / f"gradients_allsub_run{run}_{hemisphere}.npy"
np.save(gradients_path, gradients)

# 4 : Compute the watershed of each gradients to optain gradient maps
labels = watershed_by_flooding(graph, gradient_smoothed)
labels_path = config["subjects_dir"] / f"labels_allsub_run{run}_{hemisphere}.npy"
np.save(labels_path, labels)

# 5 : Visualize the watershed
visualize_brain_surface(coords_, faces_, gradient_smoothed)



In [5]:
# Full pipeline GORDON WAY

# 1 : Compute the average similartiy matrix across each subjects
sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
print('sim_matrix_sum shape', sim_matrix_sum.shape)
for i in range(1,21):
    if i == 5:
        continue # Subject 5 is missing
    # Variables path
    subject = f"{i:02d}"
    subj_dir = config["subjects_dir"] / f"sub-{subject}"

    # Define paths using pathlib
    surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
    vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
    brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"

    # Load the data
    surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                    vol_fmri_path, 
                                                    brain_mask_path)
    
    sim_matrix_sum += compute_similarty_matrix_PCA(vol_fmri_n,
                                                   surf_fmri_n)

# Save the similarty matrix
sim_matrix_path = config["subjects_dir"] / f"similarity_matrix_allsub_run{run}_{hemisphere}.npy"
np.save(sim_matrix_path, sim_matrix_sum)
    
    
# 2 : Smooth the similatry matrix
sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

# 3 : Compute the gradient of the similarty matrix
gradients = compute_gradients(graph, sim_matrix_smoothed)

# 3.1 : Smooth the gradient
gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
gradients_path = config["subjects_dir"] / f"gradients_smoothed_allsub_run{run}_{hemisphere}.npy"
np.save(gradients_path, gradients)

# 4 : Create the boundary map from the watershed
boundary_map = np.zeros_like(gradients[:,0]).astype(np.float32)
for map_idx in range(gradients.shape[1]): # loop over each columns of the matrix
    if map_idx % 10 == 0:
        print(f"Processing map {map_idx}") # Print the current map index
    boundary = (watershed_by_flooding(graph, gradients_smoothed[:,map_idx])<0)*1 # Extract the bounary from the watershed algorithm
    boundary_map += boundary
boundary_map_path = config["subjects_dir"] / f"boundary_smoothed_map_allsub_run{run}_{hemisphere}.npy"
np.save(boundary_map_path, boundary_map)

# 5 : Visualize the bounardy map
visualize_brain_surface(coords_, faces_, boundary_map)



sim_matrix_sum shape (40962, 40962)


C:\Users\Robin\AppData\Local\Temp\ipykernel_22416\3368662244.py:27: RuntimeWarning: invalid value encountered in divide
  vol_fmri_n = (vol_fmri - np.mean(vol_fmri, axis=1, keepdims=True)) / np.std(vol_fmri, axis=1, keepdims=True)
C:\Users\Robin\AppData\Local\Temp\ipykernel_22416\3368662244.py:8: RuntimeWarning: invalid value encountered in divide
  surf_fmri_n = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)
C:\Users\Robin\AppData\Local\Temp\ipykernel_22416\3368662244.py:27: RuntimeWarning: invalid value encountered in divide
  vol_fmri_n = (vol_fmri - np.mean(vol_fmri, axis=1, keepdims=True)) / np.std(vol_fmri, axis=1, keepdims=True)
c:\Users\Robin\Documents\1_EPFL\pdme\code\brain_parcellation\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Robin\Documents\1_EPFL\pdme\code\brain_parcellation\venv\Lib\site-packages\numpy\lib\_function_ba

Processing map 0
Processing map 10
Processing map 20
Processing map 30
Processing map 40
Processing map 50
Processing map 60
Processing map 70
Processing map 80
Processing map 90
Processing map 100
Processing map 110
Processing map 120
Processing map 130
Processing map 140
Processing map 150
Processing map 160
Processing map 170
Processing map 180
Processing map 190
Processing map 200
Processing map 210
Processing map 220
Processing map 230
Processing map 240
Processing map 250
Processing map 260
Processing map 270
Processing map 280
Processing map 290
Processing map 300
Processing map 310
Processing map 320
Processing map 330
Processing map 340
Processing map 350
Processing map 360
Processing map 370
Processing map 380
Processing map 390
Processing map 400
Processing map 410
Processing map 420
Processing map 430
Processing map 440
Processing map 450
Processing map 460
Processing map 470
Processing map 480
Processing map 490
Processing map 500
Processing map 510
Processing map 520
Proc

KeyboardInterrupt: 